In [1]:
from collections import Counter
from keras.models import load_model
import nltk
import numpy as np
import re

model = load_model('./model_0.9858.h5')

NGRAM = 5
MAXLEN = 39

alphabet = ['\x00',' '] + list('0123456789aAâÂăĂbBèÈềỀiIkKóÓốỐớỚqQùÙáÁấẤắẮcCfFẹẸệỆíÍjJòÒồỒờỜrRủỦàÀầẦằẰdDẽẼễỄìÌlLọỌộỘợỢsSũŨạẠậẬặẶđĐẻẺểỂịỊmMõÕỗỖỡỠtTụỤãÃẫẪẵẴeEêÊgGĩĨnNỏỎổỔởỞuUưƯảẢẩẨẳẲéÉếẾhHỉỈoOôÔơƠpPúÚứỨừỪvVýÝỵỴửỬwWỳỲzZữỮxXỷỶựỰyYỹỸ')

In [24]:
def encoderData(text):
    x = np.zeros((MAXLEN, len(alphabet)))
    for i, c in enumerate(text[:MAXLEN]):
        x[i, alphabet.index(c)] = 1
    if i < MAXLEN - 1:
        for j in range(i + 1, MAXLEN):
            x[j, 0] = 1
    return x

def decoderData(x):
    x = x.argmax(axis = -1)
    return ''.join(alphabet[i] for i in x)

In [25]:
print(encoderData('Tôi yêu bạn').shape)
print(decoderData(encoderData('Tôi yêu bạn')))

(39, 198)
Tôi yêu bạn                            


In [26]:
def nltkNGRAMS(sentence, n, maxlen):
    listNGRAMS = []
    listWords = sentence.split()
    numWords = len(listWords)

    if (numWords >= n):
        for ngram in nltk.ngrams(listWords, n):
            if len(' '.join(ngram)) <= maxlen:
                listNGRAMS.append(ngram)
    else:
        listNGRAMS.append(tuple(listWords))

    return listNGRAMS

In [27]:
def guess(ngram):
    text = ' '.join(ngram)
    preds = model.predict(np.array([encoderData(text)]))

    return decoderData(preds[0]).strip('\x00')

In [28]:
guess(('Tooi','chuoois'))

ValueError: in user code:

    File "c:\Users\DELL 7510\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2137, in predict_function  *
        return step_function(self, iterator)
    File "c:\Users\DELL 7510\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2123, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\DELL 7510\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2111, in run_step  **
        outputs = model.predict_step(data)
    File "c:\Users\DELL 7510\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 2079, in predict_step
        return self(x, training=False)
    File "c:\Users\DELL 7510\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from Non

In [17]:
def addPunctation(text, correctedText):
    listPunctation = {}

    for (i,word) in enumerate(text.split()):
        if word[0] not in alphabet or word[-1] not in alphabet:
            startPunc = ''
            for c in word:
                if c in alphabet:
                    break
                startPunc += c

            endPunc = ''
            for c in word:
                if c in alphabet:
                    break
                endPunc += c
            endPunc = endPunc[::-1]

            listPunctation[i] = [startPunc,endPunc]

    result = ''
    for (i,word) in enumerate(correctedText.split()):
        if i in listPunctation:
            result += (listPunctation[i][0] + word + listPunctation[i][1]) + ' '
        else:
            result += word + ' '

    return result.strip()

In [20]:
def correctText(text):
    newText = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)

    ngrams = list(nltkNGRAMS(newText,NGRAM,MAXLEN))
    guessedNGRAMS = list(guess(ngram) for ngram in ngrams)
    candidates = [Counter() for _ in range(len(guessedNGRAMS) + NGRAM - 1)]

    for nid, ngram in (enumerate(guessedNGRAMS)):
        for wid, word in (enumerate(re.split('\s', ngram))):
            candidates[nid+wid].update([word])

    correctedText = ' '.join(c.most_common(1)[0][0] for c in candidates if c)
    return addPunctation(text, correctedText)

In [21]:
text = input()

result = correctText(text)
print(result)

text = re.sub(r'[^' + ''.join(alphabet) + ']', '', text)
listText = text.split()

result = re.sub(r'[^' + ''.join(alphabet) + ']', '', result)
listResult = result.split()

correctWord = [(listText[i], listResult[i]) for i in range(len(listText)) if listText[i] != listResult[i]]
correctWord

1/1 [==============================] - 0s 39ms/step
oooo Vỗuoois


[('tooi', 'oooo'), ('chuoois', 'Vỗuoois')]

In [22]:
from tkinter import *
import tkinter



window = Tk()
window.title("Spell Correction")
window.geometry("800x600")
txt = Entry(window, width=100, font=('Times New Roman',12))
txt.grid(column=0,row=0)